In [ ]:
from sensor_core import SensorManager
from sensor_core import strg_manager
from sensor_core import ser_manager
import numpy as np

In [ ]:
# USER DEFINED PARAMS
ser_channel_key = ["red", "infrared", "violet"]                    # Names of serial channels
plot_channel_key = [["red", "infrared", "violet"]]                 # Names and gridplot arrangement of plot channels
commport = None                                                    # Name of comm port
baudrate = None                                                    # bits/second

In [ ]:
# Custom Serial Acquisition Function
def acquire_data(ser, frame_shape):
    """ serial acquisition function handler. All parameters are internally generated
    Must include the following parameters:
    :param ser: serial port
    :param window_size: number of frames to capture before passing to shared memory object. Default 1
    :param num_channel: number of channels to capture. 
    """
    channel_data = np.random.rand(frame_shape[1], frame_shape[2]).astype(np.float32)
    return channel_data

In [ ]:
# Instantiate Sensor Manager class
sm = SensorManager(ser_channel_key=ser_channel_key,
                   plot_channel_key=plot_channel_key,
                   commport=commport, 
                   baudrate=baudrate, 
                   dtype=np.float32,
                   data_mode='line',
                   frame_shape = (100, 10, 3), # num_points, window_size, num_channels
                   rotate_seconds=5.0,
                   start_stream_ingest=True)

In [ ]:
# Create 'update process' to handle acquired data
update_process = sm.update_data_process(save_data=False, 
                            filepath='./serial_db.sqlite3',
                            virtual_ser_port = True,
                            func=acquire_data)

# Create 'plot process' to handle plotting. Returns plot object as well
plot_process, plot = sm.setup_plotting_process()

In [ ]:
sm.static_args_dict

In [ ]:
# Start update process, start plotting process
## Recommended to use in-built start_process function to avoid Windows OS issues
sm.start_process(update_process)
sm.start_process(plot_process)

In [ ]:
# Show plot
plot.show()

In [ ]:
# add a moving average
sm.add_plot_dsp_module(
    name="ma_vis_16",
    target_algo="moving_average_filter",
    window_size=16,
    pad="percentile",
)

In [ ]:
# add another moving average
sm.add_plot_dsp_module(
    name="ma_vis_32",
    target_algo="moving_average_filter",
    window_size=32,
    pad="percentile",
)

In [ ]:
# reorder filters
sm.set_plot_dsp_order(["ma_vis_32", "ma_vis_16"])

In [ ]:
# remove filter
sm.remove_plot_dsp_module("ma_vis_32")

In [ ]:
# clear all filters
sm.clear_plot_dsp_modules()

In [ ]:
# Get metrics snapshot
sm.get_metrics()